In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from statsmodels.stats.contrast import ContrastResults

def get_contrast_matrix(model_params):
    
    # Create contrast matrix between the gaze conditions
    contrast_matrix = np.zeros((3,len(model_params)))
    contrast_matrix[0,:3] = [1, -1, 0]
    contrast_matrix[1,:3] = [0, 1,-1]
    contrast_matrix[2,:3] = [1, 0, -1]
    
    # Create a description of the contrasts 
    description = ['{}<>{}'.format(model_params.index[0], model_params.index[1]), 
                   '{}<>{}'.format(model_params.index[1], model_params.index[2]), 
                   '{}<>{}'.format(model_params.index[0], model_params.index[2]),] 
    description = [s.replace('GazeCondition[', '').replace(']', '') for s in description] # remove redundant text
    return description, contrast_matrix

def adjust_bonferroni_holm(contrasts):
    N = contrasts.pvalue.shape[0]
    c = np.arange(1, N+1, 1)
    i = np.argsort(contrasts.pvalue)[::-1]
    return np.clip(contrasts.pvalue * c[i], 0, 1)

X = data.loc[SR_mask]

## Fit mixed linear model
mixed_lm = smf.mixedlm("TrialDuration ~ 0 + GazeCondition + Block + EnvironmentName",
                 data.loc[SR_mask],
                 groups="Subject",
#                  vc_formula={"EnvironmentName": "0 + C(EnvironmentName)"},
                 re_formula="~ 1").fit()
print(mixed_lm.summary())



## Perform t-test
description, contrast_matrix = get_contrast_matrix(model_params=mixed_lm.fe_params)
results = mixed_lm.t_test(contrast_matrix, use_t=True)
results.pvalue = adjust_bonferroni_holm(results)
results = results.summary_frame()
results['description'] = description

results

# print()
# print(mixed_lm.fe_params.index[:3])



# contrasts = mdf.t_test(np.array([[1,-1,0,0,0,0], [0,1,-1,0,0,0], [1,0,-1,0,0,0]]), use_t=True)
# contrasts.pvalue = adjust_bonferroni_holm(contrasts)


# print("ICC: ", get_icc(mdf))
# # sm.qqplot(mdf.resid)
# contrasts = mdf.t_test(np.array([[1,-1,0,0,0,0], [0,1,-1,0,0,0], [1,0,-1,0,0,0]]), use_t=True)
# contrasts.pvalue = adjust_bonferroni_holm(contrasts)
# print(contrasts.summary(xname=['Ass|Ign', 'Ign|Lck', 'Ass|Lck']))
